# Imports

In [1]:
import numpy as np
import os
import pandas as pd

from joblib import dump, load
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor

# Load data

In [2]:
fp = os.path.join('..', 'data', 'processed', 'yearly_revenue.pkl')
listings = pd.read_pickle(fp)
listings.sample(5, random_state=123)

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,yearly_revenue,property_type_cleansed
966,47.66377,-122.35714,4,1.0,1.0,3.0,16209.0,Entire guest suite
1324,47.65701,-122.31653,5,2.0,2.0,2.0,25410.0,Entire apartment
1349,47.61214,-122.32980,4,1.0,0.0,2.0,24646.0,Entire condominium
813,47.70072,-122.27365,6,2.0,3.0,3.0,49810.0,Entire guesthouse
110,47.65915,-122.31587,3,1.0,1.0,2.0,18380.0,Entire apartment


In [3]:
# Train test split.
X = listings.drop('yearly_revenue', axis=1)
y = listings.yearly_revenue

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [4]:
X_train.sample(5, random_state=123)

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,property_type_cleansed
754,47.60246,-122.29388,4,1.0,2.0,2.0,Entire apartment
152,47.68075,-122.30351,2,1.0,1.0,1.0,Private room in house
1183,47.61124,-122.33063,5,1.0,1.0,1.0,Entire condominium
1648,47.57704,-122.41478,2,1.0,1.0,1.0,Entire guesthouse
1586,47.62201,-122.29275,6,3.0,4.0,4.0,Entire house


# Preliminary modeling

In [5]:
# Create pipeline that encodes categorical variables and scales numeric variables.
numerical_features = ['latitude', 'longitude']
categorical_features = X.drop(numerical_features, axis=1).columns
preprocessor = ColumnTransformer(
    transformers=[('onehot-encoder', OneHotEncoder(handle_unknown='ignore'), categorical_features),
                  ('scalar', StandardScaler(), numerical_features)])

In [6]:
%%time
# ---------------------- Instantiate Each Model ----------------------
regressors  = {
    'linear_regressor': LinearRegression(),
    'ridge_regressor': Ridge(),
    'lasso_regressor': Lasso(),
    'elastic_regressor': ElasticNet(),
    'knn_regressor':  KNeighborsRegressor(n_neighbors=5),
    'rf_regressor': RandomForestRegressor(),
    'xgb_regressor': XGBRegressor(objective='reg:squarederror')
}

pipe_dict = []


# ----- Create pipeline and evaluate preliminary performace for each model -----
pipelines = {}
metrics_df = pd.DataFrame(columns=['mean_cv'])
for i, (name, regressor) in enumerate(regressors.items()):
    pipelines[name] = Pipeline([('preprocessor', preprocessor),
                                (name, regressor)])
    
    rmse = -cross_val_score(pipelines[name],
                            X_train,
                            y_train,
                            cv=5,
                            n_jobs=-1,
                            scoring='neg_root_mean_squared_error').mean()

    metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': rmse}, index=[name]))

# -------------------------------- Stack models --------------------------------
pipelines['stack_regressor'] = StackingRegressor(estimators=[(k, v) for k, v in pipelines.items()])

rmse = -cross_val_score(pipelines['stack_regressor'],
                        X_train,
                        y_train,
                        cv=5,
                        n_jobs=-1,
                        scoring='neg_root_mean_squared_error').mean()

metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': rmse}, index=['stack_regressor']))

/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 934011274.0617065, tolerance: 80231601.00429457
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 934011274.0617065, tolerance: 80231601.00429457
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 421213604.100708, tol

CPU times: user 122 ms, sys: 93.6 ms, total: 215 ms
Wall time: 19.1 s


In [7]:
# Mean RMSE in 5 fold CV.
metrics_df

,mean_cv
linear_regressor,19769.380872
ridge_regressor,19403.490527
lasso_regressor,19699.226050
elastic_regressor,20880.123588
knn_regressor,19729.499721
rf_regressor,18818.390114
xgb_regressor,20040.712169
stack_regressor,19313.144088


# Hyperparameter tune

In [8]:
param_grid_ridge = {
    'ridge_regressor__alpha': [1.0, 1.5, 2.0, 2.25, 2.5, 2.75, 3, 1e5]
}

param_grid_lasso = {
    'lasso_regressor__alpha': [1, 10, 15, 20, 25, 30, 35]
}

param_grid_elastic = {
    'elastic_regressor__alpha': [0, 0.25, 0.5, 0.75, 1, 10, 15, 20, 25, 30, 35],
    'elastic_regressor__l1_ratio': [0.1, 0.25, 0.5, 0.75, 1]
}

param_grid_knn = {
    'knn_regressor__n_neighbors': [3, 5, 10, 15, 20],
    'knn_regressor__weights': ['uniform', 'distance'],
}

n_features = preprocessor.fit_transform(X_train).shape[1]
param_grid_rf = {
    'rf_regressor__n_estimators': [100, 150, 200],
    'rf_regressor__max_depth': [None, 2, 3, 4, 5],
    'rf_regressor__max_features': [np.sqrt(n_features)/n_features, 0.25, 0.5, 0.75]
}

param_grid_xgb = {
    'xgb_regressor__max_depth': [4, 5, 6],                   
    'xgb_regressor__learning_rate': [0.01, 0.1, 0.15, 0.2],
}

In [9]:
gdRidge = GridSearchCV(estimator=pipelines['ridge_regressor'],
                     param_grid=param_grid_ridge,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

gdLasso = GridSearchCV(estimator=pipelines['lasso_regressor'],
                     param_grid=param_grid_lasso,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

gdElastic = GridSearchCV(estimator=pipelines['elastic_regressor'],
                     param_grid=param_grid_elastic,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

gdKNN = GridSearchCV(estimator=pipelines['knn_regressor'],
                     param_grid=param_grid_knn,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

gdRf = GridSearchCV(estimator=pipelines['rf_regressor'],
                     param_grid=param_grid_rf,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

gdXGB = GridSearchCV(estimator=pipelines['xgb_regressor'],
                     param_grid=param_grid_xgb,
                     cv=5,
                     n_jobs=-1,
                     scoring='neg_root_mean_squared_error', verbose=True)

In [10]:
pipelines_grid = [gdRidge, gdLasso, gdElastic, gdKNN, gdRf, gdXGB]
for pipe in pipelines_grid:
    pipe.fit(X_train, y_train)
    rmse = -pipe.best_score_
    name = f'tuned_{pipe.estimator.steps[1][0]}'
    pipelines[name] = pipe.best_estimator_
    metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': rmse}, index=[name]))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 934011274.0617065, tolerance: 80231601.00429457
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/pipeline.py:346: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/pipeline.py:346: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/nathan/.local/share/virtualenv

Fitting 5 folds for each of 55 candidates, totalling 275 fits


/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 188085189866.69254, tolerance: 81663053.37587194
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 201623894226.31512, tolerance: 86695892.87725323
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 173427911892.75128,

/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 188085189866.69254, tolerance: 81663053.37587194
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 186953442844.13846, tolerance: 80231601.00429457
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 173427911892.75128,

/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14658704316.88086, tolerance: 84422306.93795936
  model = cd_fast.sparse_enet_coordinate_descent(


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [11]:
for model in pipelines_grid:
    print(model.best_params_)

{'ridge_regressor__alpha': 3}
{'lasso_regressor__alpha': 35}
{'elastic_regressor__alpha': 0.25, 'elastic_regressor__l1_ratio': 0.75}
{'knn_regressor__n_neighbors': 20, 'knn_regressor__weights': 'distance'}
{'rf_regressor__max_depth': None, 'rf_regressor__max_features': 0.25, 'rf_regressor__n_estimators': 200}
{'xgb_regressor__learning_rate': 0.1, 'xgb_regressor__max_depth': 6}


In [12]:
# -------------------------------- Stack models --------------------------------
tuned_stack = StackingRegressor(estimators=[('linear', pipelines['linear_regressor']),
                                      ('ridge', gdRidge.best_estimator_),
                                      ('lasso', gdLasso.best_estimator_),
                                      ('elastic', gdElastic.best_estimator_),
                                      ('knn', gdKNN.best_estimator_),
                                      ('rf', gdRf.best_estimator_),
                                      ('xgboost', gdXGB.best_estimator_)])
# Change to RidgeCV
param_grid_stack = {
    'final_estimator': [
                        Ridge(alpha=1.6e10),
                        Ridge(alpha=1.8e10),
                        Ridge(alpha=1e11),
                        Ridge(alpha=1.2e11),
                        Ridge(alpha=1.4e11),
                        ]
}

gdStack = GridSearchCV(estimator=tuned_stack,
                      param_grid=param_grid_stack,
                      cv=5,
                      n_jobs=-1,
                      scoring='neg_root_mean_squared_error', verbose=True)

gdStack.fit(X_train, y_train)

rmse = -gdStack.best_score_
pipelines['tuned_stack_regressor'] = gdStack.best_estimator_
metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': rmse}, index=['tuned_stack_regressor']))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [13]:
# Mean RMSE in 5 fold CV, with tuned models.
metrics_df

,mean_cv
linear_regressor,19769.380872
ridge_regressor,19403.490527
lasso_regressor,19699.226050
elastic_regressor,20880.123588
knn_regressor,19729.499721
rf_regressor,18818.390114
xgb_regressor,20040.712169
stack_regressor,19313.144088
tuned_ridge_regressor,19043.126892
tuned_lasso_regressor,19494.361183


# Voting regressor

In [14]:
# Find the 3 best performing models for voting regressor.
top_models = metrics_df.sort_values('mean_cv').index[:3]
metrics_df.loc[top_models]

,mean_cv
tuned_knn_regressor,18198.374954
tuned_stack_regressor,18224.827615
tuned_rf_regressor,18421.951674


In [15]:
# CV voting regressor.
estimators = [(model, pipelines[model]) for model in top_models]
pipelines['voting_regressor'] = VotingRegressor(estimators=estimators,
                                               weights=[45, 45, 10])

rmse = -cross_val_score(pipelines['voting_regressor'],
                        X_train,
                        y_train,
                        cv=5,
                        n_jobs=-1,
                        scoring='neg_root_mean_squared_error').mean()

metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': rmse}, index=['voting_regressor']))

In [16]:
metrics_df

,mean_cv
linear_regressor,19769.380872
ridge_regressor,19403.490527
lasso_regressor,19699.226050
elastic_regressor,20880.123588
knn_regressor,19729.499721
rf_regressor,18818.390114
xgb_regressor,20040.712169
stack_regressor,19313.144088
tuned_ridge_regressor,19043.126892
tuned_lasso_regressor,19494.361183


# Evaluate best models

In [17]:
# Train and evaluate the 3 best models on the entire training dataset.
final_models = metrics_df.sort_values('mean_cv').index[:3]
metrics_df.loc[final_models]

,mean_cv
voting_regressor,18086.089284
tuned_knn_regressor,18198.374954
tuned_stack_regressor,18224.827615


In [18]:
final_eval = pd.DataFrame(columns=['rmse'])
for model in final_models:
    pipelines[model].fit(X_train, y_train)
    y_pred = pipelines[model].predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    final_eval = final_eval.append(pd.DataFrame({'rmse': rmse}, index=[model]))

In [19]:
final_eval.sort_values('rmse')

,rmse
tuned_stack_regressor,25512.972716
voting_regressor,25630.961284
tuned_knn_regressor,25872.052957


# Export best model

In [20]:
best_model_name = final_eval.sort_values('rmse').iloc[0].name
fp = os.path.join('..', 'models', 'model.joblib')
dump(pipelines[best_model_name], fp)

['../models/model.joblib']